In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/glove6b200d/glove.6B.200d.txt
/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv


In [2]:
df=pd.read_csv("../input/sentiment140/training.1600000.processed.noemoticon.csv", encoding='latin-1',header=None)
df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [3]:
import matplotlib.pyplot as plt
import tensorflow as tf
import spacy
from spacy.matcher import Matcher
from spacy.tokens import Span
from spacy import displacy

In [4]:
df=df[[0,5]]
df.columns=['sentiment','tweets']
df.head()

,sentiment,tweets
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 2 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   sentiment  1600000 non-null  int64 
 1   tweets     1600000 non-null  object
dtypes: int64(1), object(1)
memory usage: 24.4+ MB


In [6]:
df.sentiment.value_counts()

4    800000
0    800000
Name: sentiment, dtype: int64

In [7]:
sent_map={0:"negative",4:"positive"}

In [8]:
df["word_count"]=df["tweets"].apply(lambda x:len(str(x).split()))
df.head()

,sentiment,tweets,word_count
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",19
1,0,is upset that he can't update his Facebook by ...,21
2,0,@Kenichan I dived many times for the ball. Man...,18
3,0,my whole body feels itchy and like its on fire,10
4,0,"@nationwideclass no, it's not behaving at all....",21


In [9]:
df["char_count"]=df["tweets"].apply(lambda x:len(str(x)))


In [10]:
def get_avg(x):
    words=x.split()
    word_len=0
    for word in words:
        word_len+=len(word)
    return word_len/len(words)    

df["avg_word_len"]=df["tweets"].apply(lambda x:get_avg(x))


In [11]:
import re
x="hi j@gmail.com and abc@abc.com"
re.findall(r'([a-zA-Z0-9+_-]+@[a-zA-Z0-9+_-]+\.[a-zA-Z0-9+_-]+)',x)

['j@gmail.com', 'abc@abc.com']

In [12]:
df["email"]=df["tweets"].apply(lambda x:re.findall(r'([a-zA-Z0-9+_-]+@[a-zA-Z0-9+_-]+\.[a-zA-Z0-9+_-]+)',x))


In [13]:
df["url"]=df["tweets"].apply(lambda x:re.findall('https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+',x))
df["tweets"]=df["tweets"].apply(lambda x:re.sub('https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'," ",x))

In [14]:
df["tweets"]=df["tweets"].apply(lambda x: re.sub(r'([a-zA-Z0-9+_-]+@[a-zA-Z0-9+_-]+\.[a-zA-Z0-9+_-]+)'," ",x))

In [15]:
df["mention"]=df["tweets"].apply(lambda x: re.findall("(^|[^@\w])@(\w{1,15})",x))
df["tweets"]=df["tweets"].apply(lambda x: re.sub("(^|[^@\w])@(\w{1,15})"," ",x)) 


In [16]:
df["hashtag"]=df["tweets"].apply(lambda x: re.findall("(^|[^#\w])#(\w{1,15})",x))
df["tweets"]=df["tweets"].apply(lambda x: re.sub("(^|[^#\w])#(\w{1,15})"," ",x)) 


In [17]:
df["tweets"]=df["tweets"].apply(lambda x: re.sub('^[a-zA-Z0-9+_-]+'," ",x))    
df

,sentiment,tweets,word_count,char_count,avg_word_len,email,url,mention,hashtag
0,0,"/2y1zl - Awww, that's a bummer. You should...",19,115,5.052632,[],[http://twitpic.com],"[(, switchfoot)]",[]
1,0,upset that he can't update his Facebook by t...,21,111,4.285714,[],[],[],[]
2,0,I dived many times for the ball. Managed to ...,18,89,3.944444,[],[],"[(, Kenichan)]",[]
3,0,whole body feels itchy and like its on fire,10,47,3.700000,[],[],[],[]
4,0,"no, it's not behaving at all. i'm mad. why a...",21,111,4.285714,[],[],"[(, nationwideclass)]",[]
...,...,...,...,...,...,...,...,...,...
1599995,4,woke up. Having no school is the best feelin...,11,56,4.090909,[],[],[],[]
1599996,4,.com - Very cool to hear old Walt interviews!...,11,78,6.090909,[],[http://blip.fm],[],[]
1599997,4,you ready for your MoJo Makeover? Ask me for...,11,57,4.181818,[],[],[],[]
1599998,4,38th Birthday to my boo of alll time!!! Tupa...,12,65,4.416667,[],[],[],[]


In [18]:
df["tweets"]=df["tweets"].apply(lambda x: " ".join(x.split()))

In [ ]:
from bs4 import BeautifulSoup
df["tweets"]=df["tweets"].apply(lambda x: BeautifulSoup(x,'lxml').get_text())

In [19]:
import unicodedata

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii
df["tweets"]=df["tweets"].apply(lambda x: remove_accents(x))

In [20]:
nlp=spacy.load('en_core_web_sm')
from spacy.lang.en.stop_words import STOP_WORDS
df["tweets"]=df["tweets"].apply(lambda x:" ".join([t for t in  str(x).split() if t not in STOP_WORDS]))
df.tail()

,sentiment,tweets,word_count,char_count,avg_word_len,email,url,mention,hashtag
1599995,4,b'woke up. Having school best feeling ever',11,56,4.090909,[],[],[],[]
1599996,4,b'.com - Very cool hear old Walt interviews! /...,11,78,6.090909,[],[http://blip.fm],[],[]
1599997,4,b'you ready MoJo Makeover? Ask details',11,57,4.181818,[],[],[],[]
1599998,4,b'38th Birthday boo alll time!!! Tupac Amaru S...,12,65,4.416667,[],[],[],[]
1599999,4,b'',5,62,11.400000,[],[],"[( , theNSPCC), ( , SparksCharity), ( , Speaki...","[( , charitytuesday)]"


In [ ]:
def make_lemma(x):
    x_list=[]
    doc=nlp(x)
    for token in doc:
        lemma=str(token.lemma_)
        if lemma=="-PRON-" or lemma=="be":
            lemma=token.text
        x_list.append(lemma)
    return " ".join(x_list)
df["tweets"]=df["tweets"].apply(lambda x:make_lemma(x))

In [ ]:
from textblob import TextBlob
df["tweets"]=df["tweets"].apply(lambda x:TextBlob(x).correct())

In [21]:
df0=df[df["sentiment"]==0].sample(2000)
df4=df[df["sentiment"]==4].sample(2000)

In [22]:
df1=df0.append(df4).sample(4000).reset_index(drop=True)
df1.head()

,sentiment,tweets,word_count,char_count,avg_word_len,email,url,mention,hashtag
0,4,b'hello good morning too',7,39,4.571429,[],[],"[(, techfun)]",[]
1,4,"b'Thanks offer. Do write publications, offline...",25,138,4.520000,[],[],"[(, joaquin_win)]",[]
2,0,"b'Oh, I joking, sounds crappy. Feel better soon.'",13,78,4.923077,[],[],"[(, KelseyMacD)]",[]
3,4,b'going beach night &amp; discovering wolverin...,11,82,6.454545,[],[],[],[]
4,4,b'some new pics sis',7,31,3.285714,[],[],[],[]


In [23]:
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}


In [24]:
def cont_to_word(x):
  if type(x) is str:
    x=x.replace('b"','')
    x=x.replace("b'","")
    for key in contractions:
      value=contractions[key]
      x=x.replace(key,value)
    return x   
  else:
    return x   
df1["tweets"]=df1["tweets"].apply(lambda x:cont_to_word(x))
df1["tweets"]=df1["tweets"].apply(lambda x:re.sub(r'[^\w\s]','',x))
df1

,sentiment,tweets,word_count,char_count,avg_word_len,email,url,mention,hashtag
0,4,hello good morning too,7,39,4.571429,[],[],"[(, techfun)]",[]
1,4,Thanks offer Do write publications offline may...,25,138,4.520000,[],[],"[(, joaquin_win)]",[]
2,0,Oh I joking sounds crappy Feel better soon,13,78,4.923077,[],[],"[(, KelseyMacD)]",[]
3,4,going beach night amp discovering wolverinesty...,11,82,6.454545,[],[],[],[]
4,4,some new pics sis,7,31,3.285714,[],[],[],[]
...,...,...,...,...,...,...,...,...,...
3995,0,s luck,4,20,4.000000,[],[],[],[]
3996,0,up camden trip baaaaa tis frigging citys fault...,19,105,4.526316,[],[],[],[]
3997,4,Global Wind Day Wind power videos empower mind...,12,84,6.000000,[],[http://sn.im],[],[]
3998,4,you want fully kit out do not you,10,56,4.600000,[],[],"[(, oliverspall)]",[]


In [25]:
y=df1["sentiment"]
from sklearn.feature_extraction.text import  CountVectorizer
cv=CountVectorizer()
text_counts=cv.fit_transform(df1["tweets"])


In [26]:
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.preprocessing import MinMaxScaler

In [27]:
sgd=SGDClassifier(n_jobs=-1,random_state=42, max_iter=200)
lgr=LogisticRegression(random_state=42,max_iter=200)
lgrcv=LogisticRegressionCV(cv=2,random_state=42, max_iter=1000)
svm=LinearSVC(random_state=42, max_iter=200)
rfc=RandomForestClassifier(n_estimators=200,random_state=42, n_jobs=-1)
clf={"SGD":sgd,"LGR":lgr,"LGRCV":lgrcv,"SVM":svm,"RFC":rfc}

In [28]:
def classify(X,y):
    scaler=MinMaxScaler(feature_range=(0,1))
    X=scaler.fit_transform(X)
    X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=101,test_size=0.2,stratify=y)
    for key in clf.keys():
        clf[key].fit(X_train,y_train)
        y_pred=clf[key].predict(X_test)
        ac=accuracy_score(y_test,y_pred)
        print(key,ac)

In [29]:
bog=pd.DataFrame(text_counts.toarray(),columns=cv.get_feature_names())
bog

,0000000000,07amin,08,0831,09,0_o,0xh5fj,10,100,1000,...,zenith,zepelin,zeppelin,zero,zimmer,zimmerframes,zloof,zoo,zuko,zune
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
classify(bog,y)

SGD 0.64
LGR 0.66375
LGRCV 0.67375
SVM 0.6475
RFC 0.67875


In [ ]:
df1

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
x=tfidf.fit_transform(df1["tweets"])
classify(pd.DataFrame(x.toarray()),y)

SGD 0.6325
LGR 0.6725
LGRCV 0.68625
SVM 0.64875
RFC 0.66
